# Normaliza texto de ementa de acordãos e pareceres prévios
Versão 2

## Introdução
Fonte de dados: arquivo 'data/raw/tce_acordaos_e_pareceres_com_ementas.csv' <br>
Normaliza o campo ementa e exporta dados para arquivo 'data/processed/tce_ementas_de_acordaos_e_pareceres.csv'

Criado por Antonio Filho em 09/11/2020


## Importação das bibliotecas

In [2]:
import pandas as pd
import numpy as np

from pathlib import Path

import os
import sys
sys.path.insert(0, os.path.abspath('../../src'))
#sys.path.insert(0, os.path.abspath('../../src/preparation'))

from acordaos_utils import clean_text
import nlp_utils

## Criar dataset de todas as decisões (acórdãos e pareceres)

In [3]:
caminho_dados = Path("../../data/raw/")
df_decisoes = pd.read_csv(caminho_dados/"tce_acordaos_e_pareceres_com_ementas.csv", sep=";")

In [4]:
df_decisoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18921 entries, 0 to 18920
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   cod_tce           18921 non-null  object 
 1   cod_processo      18921 non-null  object 
 2   tipo_processo     18921 non-null  object 
 3   cod_ato           18921 non-null  int64  
 4   tipo_ato          18921 non-null  object 
 5   peca              18921 non-null  int64  
 6   caminho           18921 non-null  object 
 7   conselheiro       15713 non-null  object 
 8   texto             18877 non-null  object 
 9   linhas            18918 non-null  object 
 10  decisao_tipo      18866 non-null  object 
 11  decisao_numero    18866 non-null  object 
 12  decisao_ano       18866 non-null  float64
 13  info_processuais  15354 non-null  object 
 14  ementa            18859 non-null  object 
 15  sumario           4893 non-null   object 
 16  tamanho_ementa    18864 non-null  float6

In [5]:
df_decisoes.head()

,cod_tce,cod_processo,tipo_processo,cod_ato,tipo_ato,peca,caminho,conselheiro,texto,linhas,decisao_tipo,decisao_numero,decisao_ano,info_processuais,ementa,sumario,tamanho_ementa
0,000132/2017,TC/000132/2017,CONTROLE SOCIAL - DENÚNCIA,1284960,ACÓRDÃO DE CÂMARA,23,/2017/06/688336/133/2/805temp/ACOCAM.pdf,NaN,ACORDÃO Nº 1647/2017\n\n\nPROCESSO TC Nº 00013...,"[{'page': 1, 'x0': 245, 'x1': 380, 'bottom': 1...",acordao,1647,2017.0,PROCESSO TC Nº 000132/2017\nDECISÃO Nº 343/17\...,Denúncia contra a Prefeitura Municipal de\nCal...,NaN,115.0
1,000189/2014,TC/000189/2014,INATIVAÇÃO - APOSENTADORIA,262333,ACÓRDÃO DE CÂMARA,10,/2014/07/623635/12/2/587temp/ACOCAM.pdf,NaN,ACÓRDÃO N.º 788/2014\n\nProcesso TC/000189/201...,"[{'page': 1, 'x0': 70, 'x1': 316, 'bottom': 13...",acordao,788,2014.0,Processo TC/000189/2014\nAssunto: Aposentadori...,EMENTA: APOSENTADORIA VOLUNTÁRIA POR\nIDADE E ...,NaN,158.0
2,000135/2020,TC/000135/2020,DOS RECURSOS - RECURSO DE RECONSIDERAÇÃO,2462530,ACÓRDÃO DE CÂMARA,15,/2020/03//770683/8/2/17032020_095623_46033325D...,NaN,ACÓRDÃO Nº 317/2020\nPROCESSO: TC/000135/2020\...,"[{'page': 1, 'x0': 245, 'x1': 377, 'bottom': 1...",acordao,317,2020.0,PROCESSO: TC/000135/2020\nASSUNTO: RECURSO DE ...,EMENTA: RECURSO DE RECONSIDERAÇÃO. FALHAS QUE\...,Sumário. Recurso de Reconsideração em face do ...,297.0
3,000271/2017,TC/000271/2017,FISCALIZAÇÃO - AUDITORIA,1389052,ACÓRDÃO DO PLENO,35,/2017/09/688475/133/3/604temp/ACOPLE.pdf,NaN,ACORDÃO Nº 2.504/17\n\nPROCESSO TC Nº 000271/2...,"[{'page': 1, 'x0': 250, 'x1': 376, 'bottom': 1...",acordao,2504,2017.0,PROCESSO TC Nº 000271/2017\nDECISÃO Nº 1.328/1...,EMENTA. INSPEÇÃO ORDINÁRIA. VERIFICAÇÃO DE\nRE...,NaN,729.0
4,000244/2017,TC/000244/2017,CONTROLE SOCIAL - DENÚNCIA,1255656,ACÓRDÃO DE CÂMARA,20,/2017/05/688448/133/2/117temp/ACOCAM.pdf,NaN,ACORDÃO Nº 1.269/2017\n\n\nPROCESSO TC Nº 0002...,"[{'page': 1, 'x0': 244, 'x1': 382, 'bottom': 1...",acordao,1269,2017.0,PROCESSO TC Nº 000244/2017\nDECISÃO Nº 261/17\...,"Denúncia contra FUESPI. Unânime, concordando\n...",NaN,201.0


### Remover colunas que não serão aproveitadas

In [6]:
df_decisoes.columns

Index(['cod_tce', 'cod_processo', 'tipo_processo', 'cod_ato', 'tipo_ato',
       'peca', 'caminho', 'conselheiro', 'texto', 'linhas', 'decisao_tipo',
       'decisao_numero', 'decisao_ano', 'info_processuais', 'ementa',
       'sumario', 'tamanho_ementa'],
      dtype='object')

In [7]:
df_decisoes = df_decisoes[['ementa']]

In [8]:
df_decisoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18921 entries, 0 to 18920
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ementa  18859 non-null  object
dtypes: object(1)
memory usage: 147.9+ KB


In [9]:
df_decisoes.head()

,ementa
0,Denúncia contra a Prefeitura Municipal de\nCal...
1,EMENTA: APOSENTADORIA VOLUNTÁRIA POR\nIDADE E ...
2,EMENTA: RECURSO DE RECONSIDERAÇÃO. FALHAS QUE\...
3,EMENTA. INSPEÇÃO ORDINÁRIA. VERIFICAÇÃO DE\nRE...
4,"Denúncia contra FUESPI. Unânime, concordando\n..."


## Normalizar texto

In [39]:
#nlp_utils.pre_processing_pipeline(df.ementa)
df_decisoes['ementa_limpa'] = df_decisoes.ementa.apply(lambda x: x.replace('\n', ' ') if isinstance(x, str) else '')
#df_decisoes['ementa_limpa'] = df_decisoes.loc[1].ementa.replace('\n', ' ')
#                                                       if isinstance(x, str) else '')
df_decisoes['ementa_limpa'] = df_decisoes.ementa_limpa.apply(lambda x: nlp_utils.pre_processing_pipeline(x) \
                                                       if isinstance(x, str) else '')

In [42]:
df_decisoes['ementa_limpa'] = df_decisoes.ementa_limpa.apply(lambda x: x.replace('ementa ', '') \
                                                             if isinstance(x, str) else '')

In [43]:
df_decisoes.head()

,ementa,ementa_limpa
0,Denúncia contra a Prefeitura Municipal de\nCal...,denuncia contra prefeitura municipal caldeirao...
1,EMENTA: APOSENTADORIA VOLUNTÁRIA POR\nIDADE E ...,aposentadoria voluntaria por idade tempo contr...
2,EMENTA: RECURSO DE RECONSIDERAÇÃO. FALHAS QUE\...,recurso reconsideracao falhas que nao ensejam ...
3,EMENTA. INSPEÇÃO ORDINÁRIA. VERIFICAÇÃO DE\nRE...,inspecao ordinaria verificacao regularidade pr...
4,"Denúncia contra FUESPI. Unânime, concordando\n...",denuncia contra fuespi unanime concordando par...


In [45]:
df_decisoes.shape

(18921, 2)

In [47]:
df_decisoes.ementa_limpa.dropna()

0        denuncia contra prefeitura municipal caldeirao...
1        aposentadoria voluntaria por idade tempo contr...
2        recurso reconsideracao falhas que nao ensejam ...
3        inspecao ordinaria verificacao regularidade pr...
4        denuncia contra fuespi unanime concordando par...
                               ...                        
18916    denuncia formulada contra sra ana celia costa ...
18917    processual tomada contas especial irregularida...
18918    prestacao contas pedido bloqueio contas descum...
18919    prestacao contas bloqueio contas ausencia pres...
18920    aposentadoria ingresso servidor servico public...
Name: ementa_limpa, Length: 18921, dtype: object

## Exportar dados para csv

In [48]:
caminho = Path("../../data/processed/tce_ementas_de_decisoes.csv")
df_decisoes.to_csv(caminho, index=False, sep=';')